# Ficha de Análise Sintática (Yacc)

Autor: Sofia Santos


## Introdução

A análise sintática, ou "*parsing*" em inglês, refere-se ao processo de analisar um texto, muitas vezes o código-fonte de um programa, para determinar a sua estrutura e significado.

Em computação, o *parsing* é usado como componente chave no processo de compilação para converter código escrito numa linguagem de programação de alto nível para código executável pela máquina. Também na linguística, o *parsing* é usado para analisar frases em linguagem natural e determinar a sua estrutura sintática.

Em Python, podemos fazer análise sintática de várias formas. A que iremos utilizar nas aulas, tal como na análise léxica, recorre ao módulo **Ply**, mais especificamente à ferramenta **Yacc**.

O **Yacc** usa uma técnica de *parsing* conhecida como *LR-parsing* (ou *shift-reduce* parsing). O L significa que o *input* é lido da esquerda para a direita e o R significa que as regras da gramática são "reduzidas" do lado direito de cada regra de derivação para o seu lado esquerdo.

Dentro do módulo `Ply`, usamos a ferramenta `yacc.py` para fazer análise sintática.

In [ ]:
import ply.yacc as yacc

Para fazer análise sintática, precisamos primeiro de fazer análise léxica. 

Assim, vamos pegar no exemplo usado na ficha anterior e estendê-lo para realizar também o *parsing* de expressões aritméticas, passando a funcionar como uma calculadora.

In [ ]:
import ply.lex as lex

tokens = (
    'NUMBER',
    'PLUS',
    'MINUS',
    'TIMES',
    'DIVIDE',
    'LPAREN',
    'RPAREN'
)

t_PLUS = r'\+'
t_MINUS = r'\-'
t_TIMES = r'\*'
t_DIVIDE = r'\/'
t_LPAREN = r'\('
t_RPAREN = r'\)'

def t_NUMBER(t):
    r'\d+'
    t.value = int(t.value)
    return t

t_ignore = ' \n\t'

def t_error(t):
    print(f"Carácter ilegal {t.value[0]}")
    t.lexer.skip(1)

lexer = lex.lex()

O parser precisa de ter acesso aos tokens. Caso o tokenizer esteja definido noutro ficheiro, devemos importar a variável.

```py
from [nome_do_tokenizer] import tokens
```
supondo que o analisador léxico anterior tinha sido gravado num ficheiro com o nome `calculadora_lex.py` a importação que teremos de fazer seria:
```py
from calculadora_lex mport tokens
```

Depois, apenas precisamos de definir as regras da gramática da Calculadora.

Se apenas pretendermos fazer a análise sintática, sem qualquer ação que não seja de validar se a frase de entrada tem a estrutura correta, isto é, se forma uma frase válida da linguagem da Calculadora, bastará escrever uma única função, com todas as regras de derivação da dita gramática, em Python na sintaxe do *ply* como se mostra a seguir (de notar que o prefixo `p_` é obrigatório no nome dessa função):

```py
def p_grammar(p):
    """
    expression : expression PLUS term
    expression : expression MINUS term
    expression : term
    term       : term TIMES factor
    term       : term DIVIDE factor
    term       : factor
    factor     : NUMBER
    factor     : LPAREN expression RPAREN
    """
```

Adicionalmente o `yacc` do **Ply** exige que se defina uma função para estabelecer o comportament do *parser* caso encontre um erro sintático na frase de entrada.

```py
def p_error(p):
  print("Erro sintático na entrada ao ler o símbolo ",p)
```

Porém caso se queira ir fazendo ações ao longo do reconhecimento léxico-sintático para produzir uma *saída* ou *resultado*, neste caso, para ir calculando o valor da expressão e escevê-lo no fim, é necessário partir a gramática em regras e definir uma função de reconhecimento para cada uma:

In [ ]:
def p_expression_plus(p):
    'expression : expression PLUS term'
    p[0] = p[1] + p[3]

def p_expression_minus(p):
    'expression : expression MINUS term'
    p[0] = p[1] - p[3]

def p_expression_term(p):
    'expression : term'
    p[0] = p[1]

def p_term_times(p):
    'term : term TIMES factor'
    p[0] = p[1] * p[3]

def p_term_div(p):
    'term : term DIVIDE factor'
    p[0] = p[1] / p[3]

def p_term_factor(p):
    'term : factor'
    p[0] = p[1]

def p_factor_num(p):
    'factor : NUMBER'
    p[0] = p[1]

def p_factor_expr(p):
    'factor : LPAREN expression RPAREN'
    p[0] = p[2]

def p_error(p):
    print("Erro sintático no input!")

Cada função corresponde ao nome de uma regra da gramática, representada por um símbolo não-terminal do lado esquerdo e uma sequência de não-terminais e *tokens* do lado direito.

Caso um não-terminal seja definido à custa de outros não-terminais, os seus valores podem ser obtidos através de `p[i]`, em que `i` é a posição do não-terminal na regra. O valor dos terminais é o valor definido no *tokenizer*.

Depois de executar a função, o valor do não-terminal à esquerda irá corresponder ao valor de `p[0]`.

Por exemplo, em

```py
def p_expression_plus(p):
    'expression : expression PLUS term'
    #   ^            ^        ^    ^
    #  p[0]         p[1]     p[2] p[3]

    p[0] = p[1] + p[3]
```

o valor de `p[2]` será "+", e o valor de `p[3]` será o valor de `p[0]` atribuído na regra `p_term_` correspondente.

A primeira regra definida determina o símbolo inicial da gramática. Neste caso é `expression`, o que significa que o *input* dado ao parser será reduzido até esta regra ser atingida ou até haver um erro do qual o parser não consiga recuperar.

Para facilitar a leitura, as regras podem também ser definidas do seguinte modo:

In [ ]:
def p_expression(p):
    '''expression : expression PLUS term
                  | expression MINUS term'''
    if p[2] == '+':
        p[0] = p[1] + p[3]
    elif p[2] == '-':
        p[0] = p[1] - p[3]

Depois de termos estas regras definidas, podemos correr o nosso *parser*.

In [ ]:
parser = yacc.yacc()

while s := input('calc > '):
   result = parser.parse(s)
   print(result)

No caso de termos uma regra como:

```py
expression : expression PLUS expression
           | expression MINUS expression
           | expression TIMES expression
           | expression DIVIDE expression
           | LPAREN expression RPAREN
           | NUMBER
``` 

se dermos ao *parser* uma expressão como "5 + 2 * 5", quando ele já tiver lido "5 + 2", não saberá se primeiro deve reduzir a adição ou ler o *token* da multiplicação, criando um conflito *shift/reduce*.

Em vez de estarmos a definir duas regras diferentes, como fizemos acima, podemos definir a precedência de cada token/terminal da seguinte forma:

In [ ]:
precedence = (
    ('left', 'PLUS', 'MINUS'),
    ('left', 'TIMES', 'DIVIDE'),
)

Agora, como o *token* da multiplicação tem maior precedência do que o da adição, vai haver um *shift* em vez de uma redução no exemplo acima, sem ambiguidades.

Para além de precedência à esquerda, podemos definir precedência à direita (útil para operadores unários, como a inversão de sinal) ou precedência não associativa, na qual estamos a dizer que não podemos combinar operadores daquele tipo.

In [ ]:
precedence = (
    ('nonassoc', 'LESSTHAN', 'GREATERTHAN'),  # Nonassociative operators (a < b < c)
    ('left', 'PLUS', 'MINUS'),
    ('left', 'TIMES', 'DIVIDE'),
    ('right', 'UMINUS'),            # Unary minus operator (-5)
)

A documentação completa do Yacc pode ser consultada em: https://ply.readthedocs.io/en/latest/ply.html#yacc

## Exercícios

### 1. Lista de compras

Constroi um analisador sintático capaz de ler uma lista de compras, organizada por secções de um supermercado (CARNE, PEIXE, LEGUMES, BEBIDAS, etc.), e de validar sintaticamente essa lista.

Exemplo de uma lista de compras em que cada produto necessário de cada secção tem um **número de código, nome, preço unitário** e **quantidade a comprar** é apresentado a seguir:

---
```
CARNE :
  - 1 :: Bife :: 10.00 :: 1;
  - 2 :: Panado :: 5.00 :: 4;
  - 3 :: Hambúrguer :: 8.00 :: 3;
  - 4 :: Almôndegas :: 7.00 :: 5;

BEBIDA :
  - 5 :: Água :: 0.40 :: 16;
  - 6 :: Sumo :: 1.50 :: 9;
  - 7 :: Refrigerante :: 1.80 :: 10;

FRUTA :
  - 8 :: Maçã :: 0.60 :: 20;
  - 9 :: Banana :: 0.50 :: 15;
  - 10 :: Laranja :: 0.80 :: 18;
  - 11 :: Pêssego :: 0.70 :: 22;
  - 12 :: Uva :: 0.90 :: 17;

LEGUMES :
  - 13 :: Alface :: 1.00 :: 25;
  - 14 :: Tomate :: 0.75 :: 23;
  - 15 :: Cebola :: 0.50 :: 28;
  - 16 :: Batata :: 0.30 :: 30;
  - 17 :: Cenoura :: 0.40 :: 26;

PASTELARIA :
  - 18 :: Bolo de Chocolate :: 3.50 :: 1;
  - 19 :: Croissant :: 1.20 :: 14;
  - 20 :: Pastel de Nata :: 1.00 :: 5;
  - 21 :: Donut :: 0.80 :: 13;
```
---
Numa segunda etape, o processador a construir a partir da GIC escrita na fase anterior, deve calcular, para além do **custo total das compras**, o **número de items** listados, e o **total de artigos** a comprar.

Numa última etape, pretende-se que enriqueça o processador anterior com um Analisador Semântico  que detete três tipos de erros de inconsistência lógica da lista de compras e que só apresente os valores calculados se a frase for *bem reconhecida* (isto é, *se fizer sentido*). Os erros semânticos a detetar são:
+ nomes de secções repetidos;
+ mais do que uma referência ao mesmo produto  (mesmo código, nome e preço);
+ uso do mesmo código para produtos diferentes (com nomes ou preços unitários diversos).

Abaixo mostra-se  uma frase com todos estes tipos de erros semânticos:

```
CARNE :
    - 1 :: Bife :: 10.00 :: 4;
    - 1 :: Bife :: 10.00 :: 1;
    - 2 :: Lombo :: 10.00 :: 4;
    - 2 :: Lombo :: 12.00 :: 1;
    - 3 :: Figado :: 2.00 :: 1;

CARNE :
    - 3 :: Costoleta :: 2.00 :: 1;
    - 1 :: Bife :: 10.00 :: 7;

PEIXE :
    - 77 :: Pescada :: 20.00 :: 2;
```


### 2. Linguagem lógica

Define um *parser* para uma linguagem de lógica simples que pode ter proposições, negações, conjunções, disjunções, implicações, equivalências e parênteses. A linguagem usa letras maiúsculas para proposições, ~ para negações, & para conjunções, | para disjunções, -> para implicações e <-> para equivalências. Assim, uma frase válida nesta linguagem poderia ser `P -> (Q & R) | ~S`. Este *parser* deve fazer parte de um programa que pede ao utilizador uma frase, faz a sua análise sintática, pergunta quais são os valores lógicos de cada proposição que apareça na frase e por fim utiliza o *parser* para determinar o valor lógico da expressão lida.

### 3. Tabela para CSV

Define um parser que converte uma tabela, escrita em formato textual, para um ficheiro CSV. 

Exemplo de uma tabela em formato textual:

--- 
```
| Nome        | Idade | Género      | Altura | Ocupação         |
|-------------|-------|-------------|--------|------------------|
| Alex        | 32    | Não-binário | 1.72   | Desenvolvimento  |
| Avery       | 29    | Feminino    | 1.85   | Marketing        |
| Casey       | 27    | Masculino   | 1.80   | Apoio ao cliente |
| David       | 40    | Masculino   | 1.76   | Gerente          |
| Emily       | 31    | Feminino    | 1.62   | Designer         |
| Frank       | 25    | Masculino   | 1.81   | Apoio ao cliente |
| Grace       | 38    | Feminino    | 1.68   | Vendas           |
| Harper      | 33    | Não-binário | 1.70   | Desenvolvimento  |
| Ivan        | 30    | Masculino   | 1.77   | Apoio ao cliente |
| Jane        | 29    | Feminino    | 1.74   | Designer         |
| Kim         | 35    | Feminino    | 1.79   | Gerente          |
| Liam        | 28    | Masculino   | 1.83   | Marketing        |
| Morgan      | 26    | Não-binário | 1.67   | Desenvolvimento  |
| Nathan      | 34    | Masculino   | 1.76   | Apoio ao cliente |
| Olivia      | 36    | Feminino    | 1.75   | Gerente          |
| Pat         | 37    | Não-binário | 1.78   | Designer         |
| Quinn       | 27    | Não-binário | 1.68   | Marketing        |
| Rachel      | 39    | Feminino    | 1.69   | Desenvolvimento  |
| Sam         | 23    | Não-binário | 1.70   | Apoio ao cliente |
| Taylor      | 31    | Não-binário | 1.81   | Vendas           |
```
---

Extra: permite que o parser consiga também converter uma tabela para um ficheiro JSON.

### 4. Árvores binárias

Considera que uma árvore binária pode ser representada segundo um formato textual em que `()` representa uma árvore vazia e `( e l r )` representa uma árvore com raiz `e`, uma árvore `l` à sua esquerda e uma árvore `r` à sua direita. 

Por exemplo, `( 5 (3 (1 () ()) ()) (8 () (10 () ())))` representa uma árvore de raiz 5, com uma árvore de raiz 3 à sua esquerda e uma árvore de raiz 8 à sua direita.

Escreva a Gramática que permite reconhecer frases como a do exemplo acima para descrever árvores binárias e, com base nessa GIC,  gere um analisador sintático que reconheça as frases cooretamente escritas.

Com base nesse *parser* desenvolva um processador capaz de converter esta representação textual numa representação em JSON da forma:

```Jason
ARVBIN =
null
| 
{
        "root": VALOR,
        "left": ARVBIN | null,
        "right": ARVBIN | null
}
```
Adicionalmente, 
cria uma  árvore binária  em Python que represente a árvore 
descrita pelo texto de entrada. 
Para isso, deves definir uma classe em Python que represente uma árvore binária.

### 5. Markdown para HTML

A partir do analisador léxico definido na ficha lex para um ficheiro Markdown, define agora um parser que converte um ficheiro Markdown num ficheiro HTML.